In [1]:
import pandas as pd
import os
def avg(l): return sum(l)/len(l)

'''
counts the length of rationales
'''
def count_rationale_length(df, batch=True):
    if batch:
        rationale_dict = {}
        for doc_evidences in df['evidences']:
            for evidence in doc_evidences:
                evidence = evidence[0]
                rationale_length = evidence['end_token'] - evidence['start_token']
                assert (rationale_length == len(evidence['text'].split()))
                doc_id = evidence['docid']
                if doc_id not in rationale_dict.keys(): rationale_dict[doc_id] = rationale_length
                else: rationale_dict[doc_id] += rationale_length
        return list(rationale_dict.values())
    else:
        rationale_len = []
        for doc_evidences in df['evidences']:
            for evidence in doc_evidences:
                rationale_length = evidence[0]['end_token'] - evidence[0]['start_token']
                assert (rationale_length == len(evidence[0]['text'].split()))
                rationale_len.append(rationale_length)
        return rationale_len

'''
counts the average length of the text files
'''
def count_text_length(doc_path):
    def count_file_length(file):
        f = open(file, 'rb')   
        text_length = 0
        for line in f.readlines():
            text_length += len(line.rstrip().split())
        return text_length

    text_dict = {}
    data_dir = directory + 'docs'
    for filename in os.listdir(data_dir):
        f = os.path.join(data_dir, filename)
        if os.path.isfile(f):
            text_dict[filename] = count_file_length(f)
    text_lengths = list(text_dict.values())
#     return avg(text_lengths)
    return text_lengths

In [23]:
'''
counts the length of rationales
'''
def count_rationale_len_multirc(df, batch=False):
    if batch:
        rationale_dict = {}
        for doc_evidences in df['evidences']:
            for evidence in doc_evidences:
                if type(evidence) is not dict:
                    evidence = evidence[0]
                    
                rationale_length = evidence['end_token'] - evidence['start_token']
                assert (rationale_length == len(evidence['text'].split()))
                
                doc_id = evidence['docid']
                if doc_id not in rationale_dict.keys(): rationale_dict[doc_id] = rationale_length
                else: rationale_dict[doc_id] += rationale_length
        return list(rationale_dict.values())
    else:
        rationale_len = []
        for doc_evidences in df['evidences']:
            for evidence in doc_evidences:
                if type(evidence) is not dict:
                    evidence = evidence[0]
                    
                rationale_length = evidence['end_token'] - evidence['start_token']
                assert (rationale_length == len(evidence['text'].split()))
                rationale_len.append(rationale_length)

                return rationale_len

In [2]:
dataset = "scifact"

In [3]:
directory = f'data/{dataset}/'
train_f_suffix = 'train'
test_f_suffix = 'test'
val_f_suffix = 'val'

In [4]:
train = pd.read_json(directory + train_f_suffix + '.jsonl', lines=True)
# train.info()

In [5]:
test = pd.read_json(directory + test_f_suffix + '.jsonl', lines=True)
# test.info()

In [6]:
val = pd.read_json(directory + val_f_suffix + '.jsonl', lines=True)
# val.info()

In [7]:
t = count_text_length(1)

In [12]:
train_dict = count_rationale_length(train)
test_dict = count_rationale_length(test)
val_dict = count_rationale_length(val)

# rationale_dict = {**train_dict, **test_dict, **val_dict}
# rationale_len = list(rationale_dict.values())
rationale_len = train_dict + test_dict + val_dict
avg(rationale_len)

78.57672849915683

In [148]:
def build_rationale_dict(df):
    rationale_dicts = []
    for i in range(len(df)):
        row = df.loc[i]
        classification = row['classification']
        for rationale in row['evidences']:
            rationale = rationale[0]
            text = rationale['text']
            doc_id = rationale['docid']
            rationale_length = rationale['end_token'] - rationale['start_token']
            assert (rationale_length == len(text.split()))
            rationale_dicts.append({'text':text, 'doc_id':doc_id, 'rationale_length':rationale_length, 'class':classification})
            
    return rationale_dicts

rationale_dicts = build_rationale_dict(train) + build_rationale_dict(test) + build_rationale_dict(val)

neg_lengths = []
pos_lengths = []
neg_ratios = []
pos_ratios = []
for rationale in rationale_dicts:
    text_len = text_dict[rationale['doc_id']]
    rationale_len = rationale['rationale_length']
    ratio = rationale_len/text_len
    if rationale['class'] == 'REFUTES': 
        neg_lengths.append(rationale_len)
        neg_ratios.append(ratio)
    else: 
        pos_lengths.append(rationale_len)
        pos_ratios.append(ratio)

In [146]:
avg(pos_ratios)

0.1428665268469391

In [54]:
avg(neg_ratio)

0.09976097922381488

In [55]:
avg(pos_ratio)

0.07092435780061514

In [155]:
text_lengths = list(text_dict.values())
rationale_lengths = list(rationale_dict.values())

mean_rationale_length = avg(rationale_lenth)
mean_text_length = avg(text_lengths)
mean_rationale_percent = sum(rationale_lengths)/sum(text_lengths)
mean_rationale_length_class = [avg(neg_lengths), avg(pos_lengths)]
mean_rationale_percent_class = [avg(neg_ratios), avg(pos_ratios)]
mean_text_length_class = 0
mean_text_length_z = mean_text_length
mean_text_length_all = mean_text_length
mean_text_length_train = 0
mean_text_length_dev = 0
mean_text_length_test = 0
mean_rationale_length_z = 0

In [161]:
stats_path = directory + 'stats.csv'
stats = pd.read_csv(stats_path)
scifact = ['SciFact', 
          ['REFUTES', 'SUPPORTS'],
          len(train),
          len(test),
          len(val),
          'RC,',
          mean_rationale_length,
          mean_text_length,
          mean_rationale_percent,
          mean_rationale_length_class,
          mean_rationale_percent_class,
          mean_text_length_class,
          mean_text_length_z,
          mean_text_length_all,
          mean_text_length_train,
          mean_text_length_dev,
          mean_text_length_test,
          mean_rationale_length_z]
stats.loc[0] = scifact
stats.to_csv(stats_path)

ValueError: Must have equal len keys and value when setting with an iterable

In [157]:
scifact

['SciFact',
 ['REFUTES', 'SUPPORTS'],
 405,
 188,
 100,
 'RC,',
 74.84439359267735,
 265.5903890160183,
 0.2818038479101867,
 [38.20302375809935, 34.74519230769231],
 [0.13399179869568198, 0.1428665268469391],
 0,
 265.5903890160183,
 265.5903890160183,
 0,
 0,
 0,
 0]